# SoniScope Prototype


In [1]:
# pip install sc3nb

In [2]:
import numpy as np
import pandas as pd

import sc3nb as scn
from sc3nb import SynthDef, Synth

import time
import math

import ipywidgets as widgets
from ipywidgets import interactive, interact
from ipywidgets import HBox, Label
from ipywidgets import IntSlider

from soniscope_jupyter import LensWidget
from soniscope_jupyter import ALLOWED_SHAPES 

### Connecting Jupyter Notebook with SuperCollider

In [2]:
# This notebook uses Thomas Herrmann's and Dennis Reinsch's tool "sc3nb" to connect to SuperCollider.
# Details here: https://sc3nb.readthedocs.io/en/latest/README.html
# If the connection works you should hear a short "bolp-blip" sound.

sc = scn.startup()

---
---

### Loading the data

Download here: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

The following text is also taken from the download website.

**Source:**

Hadi Fanaee-T

Laboratory of Artificial Intelligence and Decision Support (LIAAD), University of Porto
INESC Porto, Campus da FEUP
Rua Dr. Roberto Frias, 378
4200 - 465 Porto, Portugal

Original Source: http://capitalbikeshare.com/system-data
Weather Information: http://www.freemeteo.com
Holiday Schedule: http://dchr.dc.gov/page/holiday-schedule


**Data Set Information:**

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return back has become automatic. Through these systems, user is able to easily rent a bike from a particular position and return back at another position. Currently, there are about over 500 bike-sharing programs around the world which is composed of over 500 thousands bicycles. Today, there exists great interest in these systems due to their important role in traffic, environmental and health issues.

Apart from interesting real world applications of bike sharing systems, the characteristics of data being generated by these systems make them attractive for the research. Opposed to other transport services such as bus or subway, the duration of travel, departure and arrival position is explicitly recorded in these systems. This feature turns bike sharing system into a virtual sensor network that can be used for sensing mobility in the city. Hence, it is expected that most of important events in the city could be detected via monitoring these data.


**Attribute Information:**

day.csv has the following fields:

- instant: record index
- dteday : date
- season : season (1:winter, 2:spring, 3:summer, 4:fall)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- holiday : weather day is holiday or not (extracted from [Web Link])
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
- weathersit :
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered




For this demo, we unnormalized the dataset in order to semantically enriched the user experience.

In [ ]:
daily_file = 'bike_day_semantic.csv'
daily = pd.read_table(daily_file, sep=',')
daily

In [ ]:
numeric_cols = daily.select_dtypes(include='number').columns.values.tolist() # get possible colums for axes
numeric_cols

### Designing the Synthesiser
Here, one can change the sound that will be triggered by the individual data points during the sonification. 
We use a synth that simulates the sound of a marimba. A marimba has a clear pitch and a distinct attack/onset. Both of those qualities will be important for our sonification paradigm.

In [ ]:
Lens_Synth_def = SynthDef('Lens_Synth',
"""{ | freq = 1 , amp = 0.5, decayscale = 1, panning = 0|
    var sig, exciter;
    //exciter = WhiteNoise.ar() * EnvGen.ar(Env.perc(0.0001, 0.0001), 1);
    exciter = Impulse.ar(0);
    sig = DynKlank.ar(`[
    [1, 4, 10, 11, 12, 15], 
    [1, 0.66, 0.7, 0.36, 0.28, 0.3], 
    [1, 0.3, 0.1, 0.023, 0.021, 0.018]], exciter, freq, 0, decayscale);
    
    DetectSilence.ar(sig, 0.0001, 0.05, doneAction:2);
    Out.ar(0, Pan2.ar(sig*amp, panning));
}""").add()
#%sc Synth.new(\Lens_Synth ,["freq", 400, "amp", 0.05, "decayscale", 1.5, "panning", 0]);

Listen to your Synth:

In [ ]:
%%scs # Listen to the marimba with 200 Hz fundamental frequency
Synth.new(\Lens_Synth ,["freq", 200, "amp", 0.005, "decayscale", 1.5, "panning", 0]);

### Defining the function that will be called when the scatterplot is clicked:

In [ ]:
def on_lens_clicked(widget, x: float, y: float, filtered: pd.DataFrame, distances: pd.Series): 
    with outputBike:
      
    #----------------------------------------------------------------------------------------------
    # Store the moment of mouse click to t0
        t0 = time.time()
   
    #----------------------------------------------------------------------------------------------    
    # Duration of scanning process (equals [seconds] if, and only if the lens has size 100%)
        T_max = duration.value * widget.size
    
    #----------------------------------------------------------------------------------------------           
    # store values of user interface
        x_field = widget.x_field
        y_field = widget.y_field
        shape = widget.shape
        size = widget.size

        x_min = filtered[x_field].min()
        x_max = filtered[x_field].max()
        y_min = filtered[y_field].min()
        y_max = filtered[y_field].max()
        
        dB_range_val = dB_range.value
        dB_min = dB_range_val[0]
        dB_max = dB_range_val[1]

        midi_range_val = midi_range.value
        midi_min = midi_range_val[0]
        midi_max = midi_range_val[1]

        soni_dim_min = widget.data[soni_dim.value].min()
        soni_dim_max = widget.data[soni_dim.value].max()  

        soni_sort_dim_min = filtered[soni_sort_dim.value].min()
        soni_sort_dim_max = filtered[soni_sort_dim.value].max()
        
        if filtered.shape[0] == 1: # otherwise linlin division by zero whenever only one point in lens
            x_min = 0
            y_min = 0
            soni_dim_min = 0
            soni_sort_dim_min = 0
    
    #----------------------------------------------------------------------------------------------                 
    # pre sorting the data for faster algorithm
        if widget.shape == "circle":
            if euclidean_switch.value == "Euclidean Distance":
                dist_filt = distances[distances <= 1]
                filtered = filtered.assign(dist=dist_filt.values)
                filtered = filtered.sort_values('dist')
            elif euclidean_switch.value == "Sort":
                filtered = filtered.sort_values(soni_sort_dim.value)
            
        elif widget.shape == "yonly":
            filtered = filtered.sort_values(soni_sort_dim.value)
        
        elif widget.shape == "square" or "xonly":
            filtered = filtered.sort_values(soni_sort_dim.value)                
    #----------------------------------------------------------------------------------------------   
    if decay_switch.value == 'on': # added .value
        # Decay with respect to mean deviation between all data and lens data
        filtered_med = filtered[soni_dim.value].mean()
        data_med = widget.data[soni_dim.value].mean() # can be done outside of def
        med_diff = abs(filtered_med - data_med)       
        decay = scn.linlin(med_diff,0,widget.data[soni_dim.value].max(),1,10)
    elif decay_switch.value == 'off':
        decay = 1
        
    # ----------------------------------------------------------------------------------------------    
    # SONIFICATION    
    # the "sonifyer" will be applied to all the datapoints in the "filtered" dataframe.
    filtered.apply(sonifyer, axis=1, args = [shape, size, t0, T_max, decay, soni_dim.value, soni_dim_min, soni_dim_max, soni_sort_dim.value, soni_sort_dim_min, soni_sort_dim_max, x_field, y_field, x_min, x_max, y_min, y_max, midi_min, midi_max, dB_min, dB_max])
        
        

In [ ]:
def on_lens_released(widget): 
    # what should be done when the touch has ended or the mouse button is released 
    # (e.g., stop the sonification)
    print("lens released")

### Defining the function that will be called to send data to SuperCollider 

In [ ]:
def sonifyer(filtered, shape, size, t0, T_max, decay, soni_dim_value, soni_dim_min, soni_dim_max, soni_sort_dim_value, soni_sort_dim_min, soni_sort_dim_max, x_field, y_field, x_min, x_max, y_min, y_max, midi_min, midi_max, dB_min, dB_max):
    
    if shape == "circle": 
        
        if euclidean_switch.value == "Euclidean Distance":
            onset = t0 + scn.linlin(filtered.dist,0,1,0,T_max)
        else: 
            onset = t0 + scn.linlin(filtered[soni_sort_dim_value],soni_sort_dim_min,soni_sort_dim_max,0, T_max)
            
        panning = scn.linlin(filtered[x_field],x_min,x_max,-0.4,0.4,"minmax")
        amp = 0.01 # = -40dB
        
        ## uncomment if you want to use the cirular lens with radial volume attenuation (euclidaen distance from mouse position)
        # amp = scn.dbamp(scn.linlin(filtered.dist, 0, 1, dB_max, dB_min)) # amp not meaningfull with other sorting
        
    elif shape == "yonly":
        onset = t0 + scn.linlin(filtered[soni_sort_dim_value],soni_sort_dim_min,soni_sort_dim_max,0, duration.value)
        panning = scn.linlin(filtered[x_field],x_min,x_max,-0.4,0.4) # intuitive design decision  
        amp = 0.01 # = -40dB
        
    elif shape == "square" or "xonly":
        onset = t0 + scn.linlin(filtered[soni_sort_dim_value],soni_sort_dim_min,soni_sort_dim_max,0, T_max)
        panning = scn.linlin(filtered[x_field],x_min,x_max,-0.4,0.4)
        amp = 0.01 # = -40dB
        
    freq = scn.midicps(scn.linlin(filtered[soni_dim_value], soni_dim_min, soni_dim_max, midi_min, midi_max))

    # Now, the data is sent to SuperCollider via the open sound controll protocol (OSC)
    sc.server.bundler(onset+0.05, "/s_new", ["Lens_Synth", -1, 1, 1, "freq", freq, "amp", amp, "decayscale", decay, "panning", panning]).send()
    
    ## Optional:  Timed Queue
    ## uncomment this if you are using datasets with more then 1000 (maybe more) items. Comment the line above instead. Details about the bottleneck of OSC bundles and the Timed Queue can be found here: 
    ## https://sc3nb.readthedocs.io/en/latest/autogen/notebooks/timedqueue-examples.html#TimedQueueSC
    ## using the Timed Queue will make the code slower (but robust for more items) - we added a delay of 0.5 seconds to keep the system deterministic. You might need to increase that delay even further.
    #queue.put_bundler(onset, scn.Bundler(onset+0.5, "/s_new", ["Lens_Synth", -1, 1, 1, "freq", freq, "amp", amp, "decayscale", decay, "panning", panning]))
    

### Preparations for the user interface

In [ ]:
style = {'description_width': 'initial'}

#----------------------------------------------------
# Silders for sound parameters:
dB_range = widgets.IntRangeSlider(description='Volume Range [dB]',min=-100,max=-60,step=1,value=[-80,-60],style = style)
midi_range = widgets.IntRangeSlider(description='Pitch Range [Midi]',min=48,max=90,step=1,value=[60,90],style = style)
duration = widgets.FloatSlider(description='Scan Duration [*s]',min = 0.5, max=10, step=0.1, value = 3,style = style)

def f(dB_range,midi_range,duration):
    return [dB_range,midi_range,duration]

sliders = widgets.interactive_output(f, {'dB_range': dB_range, 'midi_range': midi_range, 'duration': duration})

#---------------------------------------------------
soni_dim = widgets.Dropdown(options=numeric_cols,
                                value='cnt',
                                description='Pitch:',
                                disabled=False)

def soni_dim_handler(change1):
    global soni_dim
    soni_dim.value = change1.new  

soni_dim.observe(soni_dim_handler, names='value')

#----------------------------------------------------
soni_sort_dim = widgets.Dropdown(options=numeric_cols,
                                       value='temp °C',
                                       description='Sort:',
                                       disabled=False)

def soni_sort_dim_handler(change2):
    global soni_sort_dim
    soni_sort_dim.value = change2.new  

soni_sort_dim.observe(soni_sort_dim_handler, names='value')

#----------------------------------------------------
decay_switch = widgets.RadioButtons(options=['off','on'],
                                    value = 'off', 
                                    description='Decay mapping',
                                    style = style)

def decay_switch_handler(change3):
    global decay_switch
    decay_switch.value = change3.new  #added .value

decay_switch.observe(decay_switch_handler, names='value')
#----------------------------------------------------
euclidean_switch = widgets.RadioButtons(options=['Sort','Euclidean Distance'],
                                        value = 'Sort',
                                        description='Optional circle sort',
                                        style = style)

def euclidean_switch_handler(change4):
    global euclidean_switch
    euclidean_switch.value = change4.new  

euclidean_switch.observe(euclidean_switch_handler, names='value')

#----------------------------------------------------
# interface text in widget
interface_text = widgets.HTML(value="""
<b>SoniScope Interface:</b> <br>
<br>
<br>
""")

# explanation text in widget
explanation_text = widgets.HTML(value="""
<b>Interface Explanation:</b> <br> 
Please choose the two data attributes to be visualized with the <b>X-Axis</b> and <b>Y-Axis</b> options. <br> 
The <b>Pitch</b> option lets you choose which data attribute will be mapped to the pitch of a marimba hit. <br>
The <b>Sort</b> option lets you choose the data attribute that will be used to sort the pitch dimension from low to high (positive polarity). <br>
You can change the size of the lens with your mouse wheel or the <b>Lens Size</b> slider.<br>
The <b>Scan Duration</b> slider lets you change the speed of the sonification [* in seconds only with a lens size of 100%].<br>
Finally, the <b>Lens Shape</b> gives you several options for different shapes of the lens.
<br>
<br> <b> Sonification paradigm:</b>
<br> The sonification will represent a third (potentially not visible) attribute via pitch. 
<br> Every item triggers a marimba sound at some point in time that is dependent on the <b>Sort</b> dimension. 
<br> Items with a low value for the <b>Sort</b> attribute will be played back earlier. 
<br>
<br> <b>Hidden options:</b>
<br> Please uncomment the respective lines in the "display" bracket in the cell above to access these options. 
<br> The <b>Pitch Range</b> slider lets you choose a lower and an upper boundary for the pitch mapping. The slider represents MIDI notes, but the pitches are not quantized to musical notes.
<br> The <b>Volume Range slider</b> will affect only the circular lens. (!! and you have to uncomment a line in the sonifyer cell) The individual auditory marks will be attenuated with increasing distance to the center of the lens.
<br> The <b>Decay mapping</b> option lets you sonify the absolute difference between the lens-mean and the global mean of the pitch attribute. All the auditory marks will have longer decay when the means differ more. 
<br> This lets you search for groups of outliers within the sonified attribute. 
<br> The <b>Optional circle sort</b> option lets you change the sorting of the circular lens to an euclidean distance sorting from the center of the circle outwards. Items that are closer to the visible center of the lens will sound earlier.
""")


### Ploting the user interface

In [ ]:
plotBike = LensWidget(daily, 'temp °C', 'hum %')

#----------------------------------------------------
sliderSize = widgets.FloatSlider(description='Lens Size', min=0.01, max=1.5, step=0.01, readout_format='.0%')

#----------------------------------------------------
radioShape = widgets.RadioButtons(options=ALLOWED_SHAPES,
    layout={'width': 'max-content'}, # If the items' names are long
    description='Lens Shape:')

#----------------------------------------------------
drpX = widgets.Dropdown(options=numeric_cols, value='temp °C', description='X-Axis:',)
drpY = widgets.Dropdown(options=numeric_cols, value='hum %', description='Y-Axis:',)
drpC = widgets.Dropdown(options=['', 'season', 'weekday', 'weathersit'], value='', description='Color:',)

#----------------------------------------------------
# connection between frontend and python
l = widgets.link((plotBike, 'size'), (sliderSize, 'value'))
l2 = widgets.link((plotBike, 'shape'), (radioShape, 'value'))
l3 = widgets.link((plotBike, 'x_field'), (drpX, 'value'))
l4 = widgets.link((plotBike, 'y_field'), (drpY, 'value'))
l5 = widgets.link((plotBike, 'color_field'), (drpC, 'value'))

#----------------------------------------------------
outputBike = widgets.Output() # for using print within the def on_clicked

#----------------------------------------------------
display(widgets.Box([
            widgets.VBox([
                interface_text,
                drpX, 
                drpY,
                drpC,
                soni_dim, # 
                soni_sort_dim,
                sliderSize,
                duration,
                #midi_range, # uncomment if you want to change pitch range interactively (run all cells above again)
                #dB_range, # uncomment if you want to change volume range for the circular lens interactively (!! also uncomment a line in the "sonifyer"-def cell above)(run all cells above again)
                radioShape,
                #decay_switch, # uncomment for the decay option (run also the cell above again)
                #euclidean_switch,  # uncomment for the eclidean distance sorting option (run also the cell above again)
            ]),
            plotBike
        ]),
        explanation_text,
        outputBike)

#----------------------------------------------------
# optional for bigger datasets
queue = scn.TimedQueueSC() # to use the Timed Queue, uncomment the last block with in the "sonifyer" cell.

#----------------------------------------------------
# You can use the SoniScpoe now :)
plotBike.on_lens_click(on_lens_clicked)
plotBike.on_lens_release(on_lens_released)

---
---
**Change the substrate size**

The lens size will not be effected by a different substrate size, event if the substrate is rectangular.
The SoniScope will always sonify the items thats lie within the visible lens.

In [ ]:
plotBike.substrate_width = 600
plotBike.substrate_height = 600

--- 
**Optional SuperCollider commands**

These commands might help you analyse/understand the sonification further.

In [ ]:
%%sc 
(
s.makeWindow; // monitor you CPU during sound generation
s.meter; // monitor the output volume 
s.scope; // monitor the waveform of the sonification
FreqScope.new; // monitor the specturm of the sonification
)

In [ ]:
# end the connection to SuperCollider
sc.exit()